In [ ]:
from twelvedata import TDClient
import pandas as pd
from datetime import datetime

# Initialize Twelve Data client
td = TDClient(apikey="your_twelvedata_api_key")  # Replace with your API key

def get_fundamentals_td(symbol):
    try:
        fundamentals = td.get_fundamentals(symbol=symbol).as_json()
        return fundamentals.get("metrics", {})
    except Exception:
        return None

def filter_graham_stocks(symbols):
    results = []
    for symbol in symbols:
        metrics = get_fundamentals_td(symbol)
        if not metrics:
            continue
        try:
            pb = float(metrics.get("price_to_book", 999))
            pe = float(metrics.get("pe_ratio", 999))
            current_ratio = float(metrics.get("current_ratio", 0))
            debt_equity = float(metrics.get("debt_equity", 999))

            if (
                pb < 1.0 and
                pe < 10 and
                current_ratio > 2.0 and
                debt_equity < 0.5
            ):
                results.append({
                    "datetime_utc": datetime.utcnow(),
                    "symbol": symbol,
                    "name": metrics.get("company_name", "N/A"),
                    "P/B": pb,
                    "P/E": pe,
                    "Current Ratio": current_ratio,
                    "Debt/Equity": debt_equity
                })
        except:
            continue
    return pd.DataFrame(results)

def filter_buffett_stocks(symbols):
    results = []
    for symbol in symbols:
        metrics = get_fundamentals_td(symbol)
        if not metrics:
            continue
        try:
            roe = float(metrics.get("return_on_equity", 0))
            fcf = float(metrics.get("free_cash_flow", 0))
            net_margin = float(metrics.get("net_profit_margin", 0))
            pe = float(metrics.get("pe_ratio", 999))
            debt_equity = float(metrics.get("debt_equity", 999))

            if (
                roe > 15 and
                fcf > 0 and
                net_margin > 10 and
                pe < 30 and
                debt_equity < 1.0
            ):
                results.append({
                    "datetime_utc": datetime.utcnow(),
                    "symbol": symbol,
                    "name": metrics.get("company_name", "N/A"),
                    "ROE": roe,
                    "Free Cash Flow": fcf,
                    "Net Margin": net_margin,
                    "P/E": pe,
                    "Debt/Equity": debt_equity
                })
        except:
            continue
    return pd.DataFrame(results)

Gold (GLD): $297.98001
Silver (SLV): $29.12000
WTI Oil (USO): $64.01000
Brent Oil (BNO): $26.08000
Natural Gas (UNG): $18.32000
Lithium (LIT): $37.47000
Zinc (PICK): $36.05000
Copper (CPER): $29.10000
Aluminum (JJU): ❌ Error - **symbol** or **figi** parameter is missing or invalid. Please provide a valid symbol according to API documentation: https://twelvedata.com/docs#reference-data
Nickel (JJN): ❌ Error - **symbol** or **figi** parameter is missing or invalid. Please provide a valid symbol according to API documentation: https://twelvedata.com/docs#reference-data


In [2]:
import asyncio
import websockets
import json
import nest_asyncio # for ipynb only

TD_API_KEY = "fe9301beb6ab4bacaf066516f7f80164"

async def stream_gold_price():
    url = "wss://ws.twelvedata.com/v1/quotes/price"
    async with websockets.connect(url) as ws:
        # Subscribe to XAU/USD
        await ws.send(json.dumps({
            "action": "subscribe",
            "params": {
                "symbols": "XAU/USD",
                "apikey": TD_API_KEY
            }
        }))
        
        # Live data loop
        while True:
            msg = await ws.recv()
            data = json.loads(msg)
            print(f"🟡 Gold Spot Price: {data.get('price')} USD")

# Run it
# asyncio.run(stream_gold_price())
# Run it (in Jupyter)
await stream_gold_price()

InvalidStatus: server rejected WebSocket connection: HTTP 200

In [6]:
# pip install twelvedata[websocket]
 
from twelvedata import TDClient
 
def on_event(e):
    print(e)
 
td = TDClient(apikey="fe9301beb6ab4bacaf066516f7f80164")
ws = td.websocket(symbols="AAPL", on_event=on_event)
ws.connect()
ws.keep_alive()

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Users/zacks/.local/share/uv/python/cpython-3.12.10-macos-aarch64-none/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/Users/zacks/Git/GitHub/value_investing/.venv/lib/python3.12/site-packages/twelvedata/websocket.py", line 203, in run
    import websocket
ModuleNotFoundError: No module named 'websocket'


In [4]:
ws.disconnect()

In [11]:
td = TDClient(apikey="fe9301beb6ab4bacaf066516f7f80164")
ws = td.websocket(symbols="XAU/USD", on_event=on_event)
ws.connect()
ws.keep_alive()

{'event': 'subscribe-status', 'status': 'error', 'success': None, 'fails': [{'symbol': 'XAU/USD'}]}


In [12]:
ws.disconnect()

In [7]:
price_data = td.quote(symbol="XAU/USD")
price = price_data.as_json()['close']

In [16]:
import requests

response = requests.get("https://api.twelvedata.com/time_series?apikey=fe9301beb6ab4bacaf066516f7f80164&interval=1min&type=stock&start_date=2025-01-30 00:00:00&end_date=2025-03-31 16:29:00&format=CSV&symbol=AAPL")

print(response.text)

datetime;open;high;low;close;volume
2025-03-31 15:59:00;222.83;222.95;221.76;222.06;135375
2025-03-31 15:58:00;222.65;222.84;222.57;222.82;46287
2025-03-31 15:57:00;222.34;222.695;222.26;222.69;81476
2025-03-31 15:56:00;222.18;222.6;222.12;222.345;64804
2025-03-31 15:55:00;222.69;222.69;221.235;222.21;88629
2025-03-31 15:54:00;222.29;222.88;222.29;222.78;79777
2025-03-31 15:53:00;222.31;222.475;222.135;222.31;62467
2025-03-31 15:52:00;222.7;222.76;222.21;222.35;83929
2025-03-31 15:51:00;224.03;224.03;222.68;222.68;97721
2025-03-31 15:50:00;222.04;225;221.96;224.02;104834
2025-03-31 15:49:00;222.23;222.28;222.04;222.065;13813
2025-03-31 15:48:00;222.22;222.29;222.015;222.24;15977
2025-03-31 15:47:00;222.36;222.37;222.22;222.26;11909
2025-03-31 15:46:00;222.355;222.375;222.1;222.33;10032
2025-03-31 15:45:00;222.07;222.36;221.945;222.335;12495
2025-03-31 15:44:00;222.28;222.36;222.07;222.09;13649
2025-03-31 15:43:00;222.16;222.33;222.15;222.265;13406
2025-03-31 15:42:00;222.2;222.29;222.0